---
**Чат-бот с веб-интерфейсом на `Gradio` и механизмом RAG с использованием библиотек `llama-cpp-python` и `langchain`**

---
RAG (Retrieval Augmented Generation) — простое и понятное объяснение статья  
https://habr.com/ru/articles/779526/

Векторные БД статьи  
https://habr.com/ru/articles/807957/  
https://habr.com/ru/articles/817173/
https://habr.com/ru/articles/784158/

RAG статьи  
https://habr.com/ru/companies/wunderfund/articles/779748/  
https://habr.com/ru/companies/bothub/articles/825850/  
https://www.promptingguide.ai/research/rag?utm_source=substack&utm_medium=email  
https://vinija.ai/nlp/RAG/  
https://www.rungalileo.io/blog/mastering-rag-how-to-architect-an-enterprise-rag-system  

Туториал по RAG от HF  
https://huggingface.co/learn/cookbook/advanced_rag

Туториалы по langchain и RAG  
https://blog.davideai.dev/series/langchain  
https://habr.com/ru/articles/729664/  

Курс Building Agentic RAG with LlamaIndex  
https://www.deeplearning.ai/short-courses/building-agentic-rag-with-llamaindex/

Компактная реализация RAG через Ollama  
https://github.com/technovangelist/videoprojects/tree/main/2024-04-04-build-rag-with-python  
Видео про RAG где она используется (RU)  
https://www.youtube.com/watch?v=DyOKAVzMWaQ


---

# Структура и описание проекта

Код итогового приложения (не для для запуска в Colab)

**Структура проекта:**

 - 📁 `models`
 - 📁 `embed_models`
 - `requirements-base.txt`
 - `requirements-cpu.txt`
 - `requirements-cuda.txt`
 - `app.py`
 - `models.py`
 - `utils.py`


**Установка и запуск:**

1) Создание и активация виртуального окружения (опционально)

 Создание окружения:
 ```
python3 -m venv env
 ```
 Активация окружения:
 - Linux
```
source env/bin/activate
```

 - Windows
```
env\Scripts\activate.bat
```

2) Установка библиотек
 - с поддержкой CPU
 ```
 pip install -r requirements-cpu.txt
 ```

 - с поддержкой CUDA
 ```
 pip install -r requirements-cuda.txt
 ```

Для установки `llama-cpp-python` на Windows с поддержкой CUDA нужно предварительно установить [Visual Studio 2022 Community](https://visualstudio.microsoft.com/ru/downloads/) и [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit-archive), как например указано в этой [инструкции](https://github.com/abetlen/llama-cpp-python/discussions/871#discussion-5812096)  
Для полной переустановки использовать команду
```
pip install --force-reinstall --no-cache-dir -r requirements.txt --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
```

Инструкции по установке [llama-cpp-python](https://github.com/abetlen/llama-cpp-python?tab=readme-ov-file#installation-configuration) и [torch](https://pytorch.org/get-started/locally/#start-locally) для других версий и систем

3) Запуск приложения
```
python3 app.py
```
 Перейти в браузере http://localhost:7860/ после того как появится надпись `Running on local URL:  http://127.0.0.1:7860`

**Содержимое файлов `requirements:`**

`requirements-base.txt`
```
gradio==5.0.1
langchain==0.3.3
langchain-community==0.3.1
langchain-huggingface==0.1.0
pdfminer.six==20240706
youtube-transcript-api==0.6.2
psutil==6.0.0
faiss-cpu==1.9.0
beautifulsoup4==4.12.3
```

`requirements-cpu.txt`
```
--extra-index-url https://download.pytorch.org/whl/cpu
--extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu
torch==2.4.1
llama_cpp_python==0.2.90
-r requirements-base.txt
```

`requirements-cuda.txt`
```
--extra-index-url https://download.pytorch.org/whl/cu125
--extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
torch==2.4.1
llama_cpp_python==0.2.90
-r requirements-base.txt
```

Если в системе установлена CUDA не 12 а 11 версии - заменить в `requirements-cuda.txt`  
`https://download.pytorch.org/whl/cu124`  
на  
`https://download.pytorch.org/whl/cu118`  
для корректной устаноки Pytorch

Папка `env` с зависимостями для CPU занимает 1,90 Gb

Содержимое `requirements.txt` для деплоя на HF  
```
--extra-index-url https://download.pytorch.org/whl/cpu
torch==2.4.1
llama_cpp_python==0.2.90
gradio==5.0.1
langchain==0.3.3
langchain-community==0.3.1
langchain-huggingface==0.1.0
pdfminer.six==20240706
youtube-transcript-api==0.6.2
psutil==6.0.0
faiss-cpu==1.9.0
beautifulsoup4==4.12.3
```
Здесь `llama_cpp_python` ставится как обычно потому что при установке через `--extra-index-url` выдает ошибку   
`Failed to load shared library '/usr/local/lib/python3.10/site-packages/llama_cpp/lib/libllama.so': libc.musl-x86_64.so.1: cannot open shared object fil`  

# Полный код приложения и запуск в Colab/Jupyter

## Установка библиотек

Установка библиотек

Страница GiHub langchain  
https://github.com/langchain-ai/langchain

In [ ]:
%%time
%%capture

# faiss-cpu or faiss-gpu
!pip install langchain langchain_community langchain_huggingface \
    pdfminer.six gradio youtube-transcript-api faiss-cpu psutil

CPU times: user 360 ms, sys: 96.2 ms, total: 456 ms
Wall time: 41.2 s


Установка библиотеки llama-cpp-python

Доки llama-cpp-python  
https://llama-cpp-python.readthedocs.io/en/latest/?badge=latest  
Страница Githib llama-cpp-python   
https://github.com/abetlen/llama-cpp-python


In [ ]:
# установка с поддержкой CPU
!pip install -q llama-cpp-python==0.2.90

# установка с поддержкой GPU CUDA
# !CMAKE_ARGS='-DGGML_CUDA=on' FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.8 MB/s eta 0:00:00


Проверка версий библиотек

In [ ]:
!pip list | grep -P 'torch|langchain|transformers|llama_cpp|gradio|youtube|huggingface-hub|pdfminer.six|faiss|beautifulsoup4'

beautifulsoup4                     4.12.3
faiss-cpu                          1.9.0
gradio                             5.0.1
gradio_client                      1.4.0
huggingface-hub                    0.25.2
langchain                          0.3.3
langchain-community                0.3.2
langchain-core                     0.3.10
langchain-huggingface              0.1.0
langchain-text-splitters           0.3.0
llama_cpp_python                   0.2.90
pdfminer.six                       20240706
sentence-transformers              3.2.0
torch                              2.4.1+cu121
torchaudio                         2.4.1+cu121
torchsummary                       1.5.1
torchvision                        0.19.1+cu121
transformers                       4.44.2
youtube-transcript-api             0.6.2


In [ ]:
# версия Python
!python -V

Python 3.10.12


## Функции

Модуль `utils.py`

In [ ]:
import csv
from pathlib import Path
from shutil import rmtree
from typing import List, Tuple, Dict, Union, Optional, Any, Iterable
from tqdm import tqdm

import psutil
import requests
from requests.exceptions import MissingSchema

import torch
import gradio as gr

from llama_cpp import Llama
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound, TranscriptsDisabled
from huggingface_hub import hf_hub_download, list_repo_tree, list_repo_files, repo_info, repo_exists, snapshot_download

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# imports for annotations
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings
from langchain_core.vectorstores import VectorStore

# from config import (
#     LLM_MODELS_PATH,
#     EMBED_MODELS_PATH,
#     GENERATE_KWARGS,
#     LOADER_CLASSES,
#     CONTEXT_TEMPLATE,
# )


# аннотации
CHAT_HISTORY = List[Optional[Dict[str, Optional[str]]]]
LLM_MODEL_DICT = Dict[str, Llama]
EMBED_MODEL_DICT = Dict[str, Embeddings]


# ===================== ДОПОЛНИТЕЛЬНЫЕ ФУНКЦИИ =======================

# получение количества свободной памяти на диске, CPU и GPU
def get_memory_usage() -> str:
    print_memory = ''

    memory_type = 'Disk'
    psutil_stats = psutil.disk_usage('.')
    memory_total = psutil_stats.total / 1024**3
    memory_usage = psutil_stats.used / 1024**3
    print_memory += f'{memory_type} Menory Usage: {memory_usage:.2f} / {memory_total:.2f} GB\n'

    memory_type = 'CPU'
    psutil_stats = psutil.virtual_memory()
    memory_total = psutil_stats.total / 1024**3
    memory_usage =  memory_total - (psutil_stats.available / 1024**3)
    print_memory += f'{memory_type} Menory Usage: {memory_usage:.2f} / {memory_total:.2f} GB\n'

    if torch.cuda.is_available():
        memory_type = 'GPU'
        memory_free, memory_total = torch.cuda.mem_get_info()
        memory_usage = memory_total - memory_free
        print_memory += f'{memory_type} Menory Usage: {memory_usage / 1024**3:.2f} / {memory_total:.2f} GB\n'

    print_memory = f'---------------\n{print_memory}---------------'
    return print_memory


# очистка списка документов
def clear_documents(documents: Iterable[Document]) -> Iterable[Document]:
    # очистка строки текста
    def clear_text(text: str) -> str:
        lines = text.split('\n')
        # брать строки, кол-во символов в которых больше 2
        lines = [line for line in lines if len(line.strip()) > 2]
        text = '\n'.join(lines).strip()
        return text

    # итерация по документам и очистка текста
    output_documents = []
    for document in documents:
        # очистка текста текущего документа
        text = clear_text(document.page_content)
        # берем документы с длиной текста более 1
        if len(text) > 10:
            # записываем очищенный текст обратно в документ и добавляем в список
            document.page_content = text
            output_documents.append(document)
    return output_documents


# ===================== ФУНКЦИИ ИНТЕРФЕЙСА =============================


# ------------- ЗАГРУЗКА LLM И ЭМБЕДИНГ МОДЕЛЕЙ ------------------------

# функция для загрузки файла по URL ссылке и отображением прогресс баров tqdm и gradio
def download_file(file_url: str, file_path: Union[str, Path]) -> None:
    response = requests.get(file_url, stream=True)
    if response.status_code != 200:
        raise Exception(f'Файл недоступен для скачивания по ссылке: {file_url}')
    # получение размера файла в байтах
    total_size = int(response.headers.get('content-length', 0))
    # прогресс бар tqdm
    progress_tqdm = tqdm(desc='Загрузка файла GGUF', total=total_size, unit='iB', unit_scale=True)
    # прогремм бар Gradio
    progress_gradio = gr.Progress()
    # счетчик кол-ва загруженных байтов
    completed_size = 0
    # открытие файла на запись
    with open(file_path, 'wb') as file:
        # загрузка файла чанками по 4Кб и запись чанка в файл
        for data in response.iter_content(chunk_size=4096):
            size = file.write(data)
            # обновление прогресс бара tqdm
            progress_tqdm.update(size)
            # # обновление прогресс бара Gradio
            completed_size += size
            desc = f'Загрузка файла GGUF, {completed_size/1024**3:.3f}/{total_size/1024**3:.3f} GB'
            progress_gradio(completed_size/total_size, desc=desc)


# загрузка и инициализация модели GGUF
def load_llm_model(model_repo: str, model_file: str) -> Tuple[LLM_MODEL_DICT, str, str]:
    # модель LLM, логи загрузки
    llm_model = None
    load_log = ''
    support_system_role = False

    # если ни один пукт не выбран в выпадающем списки с репозиториями HF
    if isinstance(model_file, list):
        load_log += 'Не выбрана модель\n'
        return {'llm_model': llm_model}, support_system_role, load_log

    # если в названии файла GGUF есть скобка (в скобках размер модели)
    if '(' in model_file:
        model_file = model_file.split('(')[0].rstrip()

    # прогресс бар Gradio
    progress = gr.Progress()
    progress(0.3, desc='Шаг 1/2: Загрузка файла GGUF')
    # итоговый путь куда надо скачать модель (файл GGUF)
    model_path = LLM_MODELS_PATH / model_file

    # если файл модели уже загружен
    if model_path.is_file():
        load_log += f'Модель {model_file} уже загружена, повторная инициализация\n'
    else:
        # попытка загрузить файл модели GGUF
        try:
            # hf_hub_download(
            #     repo_id=model_repo,
            #     filename=model_file,
            #     local_dir=LLM_MODELS_PATH,
            #     )
            # загрузка модели с прогресс баром
            gguf_url = f'https://huggingface.co/{model_repo}/resolve/main/{model_file}'
            download_file(gguf_url, model_path)
            load_log += f'Модель {model_file} загружена\n'
        # если ошибка в процессе загрузки
        except Exception as ex:
            model_path = ''
            load_log += f'Ошибка загрузки модели, код ошибки:\n{ex}\n'

    # если файл модели существует то нужно ее инициализировать
    if model_path:
        progress(0.7, desc='Шаг 2/2: Инициализация модели')
        try:
            # инициализация модели (установить verbose=True для вывода статуса инициализации модели)
            llm_model = Llama(model_path=str(model_path), n_gpu_layers=-1, verbose=False)
            # получение флага поддержки моделью системного промта
            support_system_role = 'System role not supported' not in llm_model.metadata['tokenizer.chat_template']
            # максимальный размер контекста модели
            load_log += f'Модель {model_file} инициализирована, максимальный размер контекста - {llm_model.n_ctx()} токенов\n'
        except Exception as ex:
            load_log += f'Ошибка инициализации модели, код ошибки:\n{ex}\n'

    llm_model = {'llm_model': llm_model}
    return llm_model, support_system_role, load_log


# загрузка и инициализация модели эмбедингов
def load_embed_model(model_repo: str) -> Tuple[Dict[str, HuggingFaceEmbeddings], str]:
    # модель эмбедингов и логи загрузки
    embed_model = None
    load_log = ''

    # если ни один пукт не выбран в выпадающем списки с репозиториями HF
    if isinstance(model_repo, list):
        load_log = 'Не выбрана модель'
        return embed_model, load_log

    # прогресс бар Gradio
    progress = gr.Progress()
    # при скачивании моделей из HF их папки переименовываются из USER_NAME/REPO_NAME в USER_NAME_REPO_NAME
    folder_name = model_repo.replace('/', '_')
    # итоговый путь куда надо скачать модель (папка с моделью)
    folder_path = EMBED_MODELS_PATH / folder_name
    # если папка с моделью уже загружена
    if Path(folder_path).is_dir():
        load_log += f'Повторная инициализация модели {model_repo} \n'
    # иначе модель нужно загрузить
    else:
        progress(0.5, desc='Шаг 1/2: Загрузка репозитория модели')
        snapshot_download(
            repo_id=model_repo,
            local_dir=folder_path,
            ignore_patterns='*.h5',
        )
        load_log += f'Модель {model_repo} загружена \n'

    # инициализация модели эмбедингов
    progress(0.7, desc='Шаг 2/2: Инициализация модели')
    model_kwargs = {'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
    embed_model = HuggingFaceEmbeddings(
        model_name=str(folder_path),
        model_kwargs=model_kwargs,
        # encode_kwargs={'normalize_embeddings': True},
        )
    load_log += f'Модель эмбедингов {model_repo} инициализирована\n'
    load_log += f'Загрузите документы и инициализируйте БД заново\n'
    embed_model = {'embed_model': embed_model}
    return embed_model, load_log


# добавление ноового репозитория HF new_model_repo к текущему списку model_repos
def add_new_model_repo(new_model_repo: str, model_repos: List[str]) -> Tuple[gr.Dropdown, str]:
    load_log = ''
    repo = new_model_repo.strip()
    # проверка что переданный репозиторий не пустая строка
    if repo:
        repo = repo.split('/')[-2:]
        # проверка что репозиторий в правильном формате USER_NAME/REPO_NAME
        if len(repo) == 2:
            repo = '/'.join(repo).split('?')[0]
            # проверка что репозитория еще нет в списке готовых
            if repo_exists(repo) and repo not in model_repos:
                # доп проверка что файлы GGUF есть в репозитории для LLM
                # if any([file_name.endswith('.gguf') for file_name in list_repo_files(repo)]):

                # добавление репозитория в список существующих
                model_repos.insert(0, repo)
                load_log += f'Репозиторий модели {repo} успешно добавлен\n'
            else:
                load_log += 'Неверное название репозитория HF или модель уже есть в списке\n'
        else:
            load_log += 'Неверная ссылка на репозиторий HF\n'
    else:
        load_log += 'Пустая строка в поле репозитория HF\n'
    # вернуть выпадающий список с репозиториями HF
    model_repo_dropdown = gr.Dropdown(choices=model_repos, value=model_repos[0])
    return model_repo_dropdown, load_log


# получить список моделей GGUF из репозитория HF
def get_gguf_model_names(model_repo: str) -> gr.Dropdown:
    # получить инфо о всех файлах в репозитории
    repo_files = list(list_repo_tree(model_repo))
    # получить спиосок инфо j файлач GGUF
    repo_files = [file for file in repo_files if file.path.endswith('.gguf')]
    # получить названия файло GGUF и их размер
    model_paths = [f'{file.path} ({file.size / 1000 ** 3:.2f}G)' for file in repo_files]

    # вернуть выпадающий спискок с названиями и размерами файлов GGUF
    model_paths_dropdown = gr.Dropdown(
        choices=model_paths,
        value=model_paths[0],
        label='Файл модели GGUF',
        )
    return model_paths_dropdown


# удаление файлов и папок моделей для очистки места кроме текущей модели gguf_filename
def clear_llm_folder(gguf_filename: str) -> None:
    if gguf_filename is None:
        gr.Info(f'Не выбрано название файла модели которую не нужно удалять')
        return
    # если в названии файла GGUF есть скобка (в скобках размер модели который сами добавили в get_gguf_model_names())
    if '(' in gguf_filename:
        # то убрать размер модели в названии чтобы это было названием файла GGUF
        gguf_filename = gguf_filename.split('(')[0].rstrip()
    # удаление всех файлов кроме текущего
    for path in LLM_MODELS_PATH.iterdir():
        if path.name == gguf_filename:
            continue
        if path.is_file():
            path.unlink(missing_ok=True)
    gr.Info(f'Все файлы удалены из директории {LLM_MODELS_PATH} кроме {gguf_filename}')


# удаление папок моделей для очистки места кроме текущей модели model_folder_name
def clear_embed_folder(model_repo: str) -> None:
    # чтобы удалить все модели кроме тукущей, она должна быть выбрана
    if model_repo is None:
        gr.Info(f'Не выбрано название модели которую не нужно удалять')
        return
    # при скачивании моделей из HF их папки переименовываются из USER_NAME/REPO_NAME в USER_NAME_REPO_NAME
    model_folder_name = model_repo.replace('/', '_')
    # удаление всех папок с моделями кроме текущей
    for path in EMBED_MODELS_PATH.iterdir():
        if path.name == model_folder_name:
            continue
        if path.is_dir():
            rmtree(path, ignore_errors=True)
    gr.Info(f'Все директории удалены из директории {EMBED_MODELS_PATH} кроме {model_folder_name}')


# # дополнительная очистка видео и оперативной памяти в Colab
# def clear_memory() -> None:
#     gc.collect()
#     torch.cuda.empty_cache()


# ------------------------ YOUTUBE ------------------------

# функция проверки доступности субтитров, если доступны ручные или автоматические - возвращает True и логи
# если субтитры недоступны - возвращает False и логи
def check_subtitles_available(yt_video_link: str, target_lang: str) -> Tuple[bool, str]:
    # извлечение ID видео из полной ссылки видео на YouTube
    video_id = yt_video_link.split('watch?v=')[-1].split('&')[0]
    # строка с логами
    load_log = ''
    # статус доступности субтитров
    available = True
    try:
        # доступные языки субтитров для текущего видео
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        try:
            # поиск языка target_lang в доступных языках
            transcript = transcript_list.find_transcript([target_lang])
            # проверка что субтитры автоматические или ручные
            if transcript.is_generated:
                load_log += f'Будут загружены автоматические субтитры, ручные недоступны для видео {yt_video_link}\n'
            else:
                load_log += f'Будут загружены ручные субтитры для видео {yt_video_link}\n'
        # если субтитры для видео есть но нет субтитров на желаемом языке
        except NoTranscriptFound:
            load_log += f'Язык субтитров {target_lang} недоступен для видео {yt_video_link}\n'
            available = False
    # если никаких субтитров для видео нет
    except TranscriptsDisabled:
        load_log += f'Нет субтитров для видео {yt_video_link}\n'
        available = False
    return available, load_log


# ------------- ЗАГРУЗКА ДОКУМЕНТОВ ДЛЯ RAG ------------------------

# извлечение документов (в формате langchain Documents) из загруженных файлов
def load_documents_from_files(upload_files: List[str]) -> Tuple[List[Document], str]:
    # логи загрузки и список для загруженных документов
    load_log = ''
    documents = []
    # итерация по путям до файлов
    for upload_file in upload_files:
        # получение расширения файла
        file_extension = f".{upload_file.split('.')[-1]}"
        # если файл расширения есть в словаре с лоадерами LOADER_CLASSES
        if file_extension in LOADER_CLASSES:
            # извлекаем соотвествующий раширению лоадер
            loader_class = LOADER_CLASSES[file_extension]
            loader_kwargs = {}
            # если это csv то дополнительно получить его разделитель
            if file_extension == '.csv':
                with open(upload_file) as csvfile:
                    delimiter = csv.Sniffer().sniff(csvfile.read(4096)).delimiter
                loader_kwargs = {'csv_args': {'delimiter': delimiter}}
            try:
                # получаем документы из файлов
                load_documents = loader_class(upload_file, **loader_kwargs).load()
                documents.extend(load_documents)
            except Exception as ex:
                load_log += f'Ошибка загрузки файла {upload_file}\n'
                load_log += f'Код ошибки: {ex}\n'
                continue
        else:
            load_log += f'Неподдерживаемый формат файла {upload_file}\n'
            continue
    return documents, load_log


# извлечение документов (в формате langchain Documents) из WEB ссылок
def load_documents_from_links(
        web_links: str,  # ссылки на web-сайты или на видео YouTube
        subtitles_lang: str,  # желаемый язык субтитров на YouTube
        ) -> Tuple[List[Document], str]:

    # логи загрузки, список для документов и параметры для лоадера
    load_log = ''
    documents = []
    loader_class_kwargs = {}
    # фильтрация пустых строк
    web_links = [web_link.strip() for web_link in web_links.split('\n') if web_link.strip()]

    # итераци по переданным ссылкам
    for web_link in web_links:
        # ----------------- ссылка на YouTube ---------------------------------
        if 'youtube.com' in web_link:
            # проверка что субтитры на выбранном языке subtitles_lang доступны в видео web_link
            available, log = check_subtitles_available(web_link, subtitles_lang)
            load_log += log
            # если субтитры недоступны - пропускаем ссылку
            if not available:
                continue
            # инициализация YouTubeLoader с параметром языка субтитров
            loader_class = LOADER_CLASSES['youtube'].from_youtube_url
            loader_class_kwargs = {'language': subtitles_lang}

        # ----------------- ссылка не на YouTube ------------------------------
        else:
            # инициализация WebBaseLoader
            loader_class = LOADER_CLASSES['web']
        try:
            # проверка что сайт доступен для парсинга с помощью python requests
            if requests.get(web_link).status_code != 200:
                load_log += f'Ссылка недоступна для Python requests: {web_link}\n'
                continue
            # если доступен то загружаем документ с текстом из web ссылки
            load_documents = loader_class(web_link, **loader_class_kwargs).load()
            if len(load_documents) == 0:
                load_log += f'Фрагменты текста не были найдены по ссылке: {web_link}\n'
                continue
            documents.extend(load_documents)
        except MissingSchema:
            # если ссылка неверная
            load_log += f'Неверная ссылка: {web_link}\n'
            continue
        except Exception as ex:
            # если какая то другая ошибка
            load_log += f'Ошибка загрузки web лоадером данных по ссылке: {web_link}\n'
            load_log += f'Код ошибки: {ex}\n'
            continue
    return documents, load_log


# загрузка файлов и формирование документов и БД
def load_documents_and_create_db(
        upload_files: Optional[List[str]],  # пути до файлов, загружаемые через gr.File()
        web_links: str,  # ссылки на web-сайты или на видео YouTube
        subtitles_lang: str,  # желаемый язык субтитров на YouTube
        chunk_size: int,  # кол-во символов во фрагменте текста
        chunk_overlap: int,  # длина перекрытия фрагментов текста
        embed_model_dict: EMBED_MODEL_DICT,  # словарь с моделью эмбедингов
        ) -> Tuple[List[Document], Optional[VectorStore], str]:

    # логи загрузки, список для документов из всех источников, БД и прогресс бар Gradio
    load_log = ''
    all_documents = []
    db = None
    progress = gr.Progress()

    # проверка если модель эмбедингов отсуствует по каким то причинам
    embed_model = embed_model_dict.get('embed_model')
    if embed_model is None:
        load_log += 'Модель эмбедингов не инициализирована, БД не может быть создана'
        return all_documents, db, load_log

    # если не переданы ни пути до файлов ни ссылки
    if upload_files is None and not web_links:
        load_log = 'Не выбраны файлы или ссылки'
        return all_documents, db, load_log

    # загрузка документов из файлов
    if upload_files is not None:
        progress(0.3, desc='Шаг 1/2: Загрузка документов из файлов')
        docs, log = load_documents_from_files(upload_files)
        all_documents.extend(docs)
        load_log += log

    # загрузка документов по ссылкам
    if web_links:
        progress(0.3 if upload_files is None else 0.5, desc='Шаг 1/2: Загрузка документов по ссылкам')
        docs, log = load_documents_from_links(web_links, subtitles_lang)
        all_documents.extend(docs)
        load_log += log

    # если не загружено ни одного документа
    if len(all_documents) == 0:
        load_log += 'Загрузка прервана так как не было извлечено ни одного документа\n'
        load_log += 'Режим RAG не может быть активирован'
        return all_documents, db, load_log

    load_log += f'Загружено документов: {len(all_documents)}\n'

    # разделить документы на фрагменты и очистка фрагментов
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        # distance_strategy=DistanceStrategy.COSINE,
    )
    documents = text_splitter.split_documents(all_documents)
    documents = clear_documents(documents)
    load_log += f'Документы разделены, кол-во фрагментов текста: {len(documents)}\n'

    # инициализация БД
    progress(0.7, desc='Шаг 2/2: Инициализация БД')
    db = FAISS.from_documents(documents=documents, embedding=embed_model, )
    load_log += 'БД инициализирована, режим RAG активирован и может быть дективирован на вкладке Chatbot'
    return documents, db, load_log


# ------------------ ФУНКЦИИ ЧАТ БОТА ------------------------

# добавление сообщение пользователя в окошко чат бота
def user_message_to_chatbot(user_message: str, chatbot: CHAT_HISTORY) -> Tuple[str, CHAT_HISTORY]:
    # if user_message:
    # chatbot.append([user_message, None])  # gradio < 5
    chatbot.append({'role': 'user', 'metadata': {'title': None}, 'content': user_message})
    return '', chatbot


# форматирование промта с добавлением контекста, если БД доступна и режим RAG активирован
# иначе формирование промта без контекста - для простого общения с ботом
def update_user_message_with_context(
        chatbot: CHAT_HISTORY,  # объект чатбота - список списков реплик юзера и бота
        rag_mode: bool,  # режим RAG - включен или отключен
        db: VectorStore,  # векторная БД
        k: Union[int, str],  # кол-во релевантных запросу пользователя фрагментов текста для контекста
        score_threshold: float,  # порого по которому ищутся релевантные фрагменты текста для контекста
        ) -> Tuple[str, CHAT_HISTORY]:

    # текущее сообщение пользователя
    user_message = chatbot[-1]['content']
    # если RAG не используется то обогащенный контекстом промт пользователя будет пустой
    user_message_with_context = ''

    # если БД готова и включен режим RAG то ищем релевантные доки и добавляем в промт
    if db is not None and rag_mode and user_message.strip():
        # использовать все документы, скор которых больше score_threshold
        if k == 'all':
            k = len(db.docstore._dict)
        # поиск релевантных фрагментов текста
        docs_and_distances = db.similarity_search_with_relevance_scores(
            user_message,
            k=k,
            score_threshold=score_threshold,
            )
        # если релевантные документы найдены
        if len(docs_and_distances) > 0:
            # формирование контекста из релевантных фрагментов текста
            retriever_context = '\n\n'.join([doc[0].page_content for doc in docs_and_distances])
            # обогащение сообщения юзера контекстом
            user_message_with_context = CONTEXT_TEMPLATE.format(
                user_message=user_message,
                context=retriever_context,
                )
    return user_message_with_context


# генерация текста моделью
def get_llm_response(
        chatbot: CHAT_HISTORY,  # список переписок пользователя и бота, который отображается в окошке чат-бота
        llm_model_dict: LLM_MODEL_DICT,  # словарь с модель llama-cpp-python
        user_message_with_context: str,  # текущеее сообщение пользователя обогащенное контекстом
        rag_mode: bool,  # режим RAG - включен или отключен
        system_prompt: str,  # системный промт
        support_system_role: bool,  # поддержиывает ли модель системный промт
        history_len: int,  # кол-во предыдущих сообщений которые учитываются в истории
        do_sample: bool,  # использовать ли случайное семплирование при генерации ответа моделью
        *generate_args,  # параметры генерации ответа модели
        ) -> CHAT_HISTORY:

    # получение модели
    llm_model = llm_model_dict.get('llm_model')
    if llm_model is None:
        gr.Info('Model not initialized')
        yield chatbot
        return

    # обновление словаря параметров генерации текста
    gen_kwargs = dict(zip(GENERATE_KWARGS.keys(), generate_args))
    gen_kwargs['top_k'] = int(gen_kwargs['top_k'])
    # отключение случайного семплирования если do_sample=False
    if not do_sample:
        gen_kwargs['top_p'] = 0.0
        gen_kwargs['top_k'] = 1
        gen_kwargs['repeat_penalty'] = 1.0

    # получение текущего сообщения пользователя
    user_message = chatbot[-1]['content']
    # если сообщение пользователя пустое - возвращаем текущее окно чат бота
    if not user_message.strip():
        # для этого сначала генерируем переписку до текущего пустого сообщения юзера
        yield chatbot[:-1]
        # затем выходим из функции генератора через return None
        return

    # если включен режим RAG
    if rag_mode:
         # если запрос пользователя с контекстом не пустой то это и будет запрос пользователя
        if user_message_with_context:
            user_message = user_message_with_context
        # если обогащенный контекстом промт отсутствует и режим RAG активен то вернуть текущее окно бота без изменений
        else:
            gr.Info((
                f'Релевантные запросу документы не найдены, генерация в режиме RAG невозможна.\n'
                f'Попробуйте уменьшить searh_score_threshold или отключите режим RAG для обычной генерации'
                ))
            yield chatbot[:-1]
            return

    # список с переписками юзера и бота для подачи в модель
    messages = []

    # добавление системного промта если он есть и поддерживается моделью
    if support_system_role and system_prompt:
        messages.append({'role': 'system', 'metadata': {'title': None}, 'content': system_prompt})

    # добавление истории переписки в промт (по 2 сообщения - юзера и бота)
    if history_len != 0:
        messages.extend(chatbot[:-1][-(history_len*2):])

    # добавление текущего сообщения пользователя
    messages.append({'role': 'user', 'metadata': {'title': None}, 'content': user_message})

    # формирование генератора для генерации ответа моделью с форматированием промта
    stream_response = llm_model.create_chat_completion(
        messages=messages,
        stream=True,
        **gen_kwargs,
        )

    # обработка исключений при генерации ответа моделью, например неподдерживаемый размер контекста
    try:
        # пустую строку будем конкатенировать с токенами ответа модели
        chatbot.append({'role': 'assistant', 'metadata': {'title': None}, 'content': ''})
        # итерация и генерация токенов ответа модели в цикле
        for chunk in stream_response:
            token = chunk['choices'][0]['delta'].get('content')
            if token is not None:
                chatbot[-1]['content'] += token
                yield chatbot
    except Exception as ex:
        gr.Info(f'Ошибка при генерации ответа, код ошибки: {ex}')
        yield chatbot[:-1]
        return

## Конфиг

Модуль `congig.py`

In [ ]:
from pathlib import Path

# document loaders
from langchain_community.document_loaders import (
    CSVLoader,
    PDFMinerLoader,
    PyPDFLoader,
    TextLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
    WebBaseLoader,
    YoutubeLoader,
    DirectoryLoader,
)


# классы langchain для извлечения текста из различных источников
LOADER_CLASSES = {
    '.csv': CSVLoader,
    '.doc': UnstructuredWordDocumentLoader,
    '.docx': UnstructuredWordDocumentLoader,
    '.html': UnstructuredHTMLLoader,
    '.md': UnstructuredMarkdownLoader,
    '.pdf': PDFMinerLoader,
    '.ppt': UnstructuredPowerPointLoader,
    '.pptx': UnstructuredPowerPointLoader,
    '.txt': TextLoader,
    'web': WebBaseLoader,
    'directory': DirectoryLoader,
    'youtube': YoutubeLoader,
}

# пример видео с автоматически сгенерированными субтитрами
# https://www.youtube.com/watch?v=CFVABT8wtl4
# пример видео с обычными субтитрами
# https://www.youtube.com/watch?v=EEGk7gHoKfY

# языки для субтитров YouTube
SUBTITLES_LANGUAGES = ['ru', 'en']

# шаблон промта при условии контекста
CONTEXT_TEMPLATE = '''Ответь на вопрос при условии контекста.

Контекст:
{context}

Вопрос:
{user_message}

Ответ:'''

# словарь для конфига генерации текста
GENERATE_KWARGS = dict(
    temperature=0.2,  # температура для софтмакса
    top_p=0.95,  # сумма вероятностей токенов из которых нужно выбирать следующий токен
    top_k=40,  # из скольки максимально вероятных токенов выбирать следующий токен
    repeat_penalty=1.0,  # штраф модели за повторения
    )

# пути до моделей LLM и эмбедингов
LLM_MODELS_PATH = Path('models')
EMBED_MODELS_PATH = Path('embed_models')
LLM_MODELS_PATH.mkdir(exist_ok=True)
EMBED_MODELS_PATH.mkdir(exist_ok=True)

# доступные при запуске приложения LLM модели в формате GGUF
LLM_MODEL_REPOS = [
    # https://huggingface.co/bartowski/gemma-2-2b-it-GGUF
    'bartowski/gemma-2-2b-it-GGUF',
    # https://huggingface.co/bartowski/Qwen2.5-3B-Instruct-GGUF
    'bartowski/Qwen2.5-3B-Instruct-GGUF',
    # https://huggingface.co/bartowski/Qwen2.5-1.5B-Instruct-GGUF
    'bartowski/Qwen2.5-1.5B-Instruct-GGUF',
    # https://huggingface.co/bartowski/openchat-3.6-8b-20240522-GGUF
    'bartowski/openchat-3.6-8b-20240522-GGUF',
    # https://huggingface.co/bartowski/Mistral-7B-Instruct-v0.3-GGUF
    'bartowski/Mistral-7B-Instruct-v0.3-GGUF',
    # https://huggingface.co/bartowski/Llama-3.2-3B-Instruct-GGUF
    'bartowski/Llama-3.2-3B-Instruct-GGUF',
]

# доступные при запуске приложения модели эмбедингов
EMBED_MODEL_REPOS = [
    # https://huggingface.co/sergeyzh/rubert-tiny-turbo  # 117 MB
    'sergeyzh/rubert-tiny-turbo',
    # https://huggingface.co/cointegrated/rubert-tiny2  # 118 MB
    'cointegrated/rubert-tiny2',
    # https://huggingface.co/cointegrated/LaBSE-en-ru  # 516 MB
    'cointegrated/LaBSE-en-ru',
    # https://huggingface.co/sergeyzh/LaBSE-ru-turbo  # 513 MB
    'sergeyzh/LaBSE-ru-turbo',
    # https://huggingface.co/intfloat/multilingual-e5-large  # 2.24 GB
    'intfloat/multilingual-e5-large',
    # https://huggingface.co/intfloat/multilingual-e5-base  # 1.11 GB
    'intfloat/multilingual-e5-base',
    # https://huggingface.co/intfloat/multilingual-e5-small  # 471 MB
    'intfloat/multilingual-e5-small',
    # https://huggingface.co/intfloat/multilingual-e5-large-instruct  # 1.12 GB
    'intfloat/multilingual-e5-large-instruct',
    # https://huggingface.co/sentence-transformers/all-mpnet-base-v2  # 438 MB
    'sentence-transformers/all-mpnet-base-v2',
    # https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2  # 1.11 GB
    'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
    # https://huggingface.co/ai-forever?search_models=ruElectra  # 356 MB
    'ai-forever/ruElectra-medium',
    # https://huggingface.co/ai-forever/sbert_large_nlu_ru  # 1.71 GB
    'ai-forever/sbert_large_nlu_ru',
]

## Приложение

Модуль `app.py`

Запуск приложения

In [ ]:
from typing import List, Optional

import gradio as gr
from langchain_core.vectorstores import VectorStore

# расскоментировать импорты при запуске не в Colab

# from config import (
#     LLM_MODEL_REPOS,
#     EMBED_MODEL_REPOS,
#     SUBTITLES_LANGUAGES,
#     GENERATE_KWARGS,
# )

# from utils import (
#     load_llm_model,
#     load_embed_model,
#     load_documents_and_create_db,
#     user_message_to_chatbot,
#     update_user_message_with_context,
#     get_llm_response,
#     get_gguf_model_names,
#     add_new_model_repo,
#     clear_llm_folder,
#     clear_embed_folder,
#     get_memory_usage,
# )


# =============== ФУНКЦИИ ИНИЦИАЛИЗАЦИИ КОМПОНЕНТ ИНТЕРФЕЙСА ============

# функция получения параметров настройки RAG
def get_rag_settings(rag_mode: bool, render: bool = True):
    # кол-во релевантных фрагментов текста для поиска в БД
    k = gr.Radio(
        choices=[1, 2, 3, 4, 5, 'all'],
        value=2,
        label='Количество релевантных документов для поиска',
        visible=rag_mode,
        render=render,
        )
    # порог для поиска релевантных фрагментов текста от 0 до 1 (чем ниже тем больше документов будет найдено)
    score_threshold = gr.Slider(
        minimum=0,
        maximum=1,
        value=0.5,
        step=0.05,
        label='relevance_scores_threshold',
        visible=rag_mode,
        render=render,
        )
    return k, score_threshold


# получить текст текущего промта с контекстом
def get_user_message_with_context(text: str, rag_mode: bool) -> gr.component:
    # настройка кол-ва строк в отображаемом окне для удобства
    num_lines = len(text.split('\n'))
    max_lines = 10
    num_lines = max_lines if num_lines > max_lines else num_lines
    return gr.Textbox(
        text,
        visible=rag_mode,
        interactive=False,
        label='User Message With Context',
        lines=num_lines,
        )


# получение окошка для системного промта. если interactive=True то можно вводить системный промт
# примеры системных промтов:
# Отвечай на вопросы максимально кратко, если не знаешь ответ - говори "не знаю".
# Если в вопросе есть хоть малейший намек на политику, насилие, ругательства, ответь так: "На эту тему я не могу говорить".
def get_system_prompt_component(interactive: bool) -> gr.Textbox:
    value = '' if interactive else 'System prompt is not supported by this model'
    return gr.Textbox(value=value, label='System prompt', interactive=interactive)


# компоненты настройки конфига генерации текста
def get_generate_args(do_sample: bool) -> List[gr.component]:
    # если do_sample включен (элемент gr.Checkbox() активен) то отображать слайдера с параметрами генерации
    generate_args = [
        gr.Slider(minimum=0.1, maximum=3, value=GENERATE_KWARGS['temperature'], step=0.1, label='temperature', visible=do_sample),
        gr.Slider(minimum=0.1, maximum=1, value=GENERATE_KWARGS['top_p'], step=0.01, label='top_p', visible=do_sample),
        gr.Slider(minimum=1, maximum=50, value=GENERATE_KWARGS['top_k'], step=1, label='top_k', visible=do_sample),
        gr.Slider(minimum=1, maximum=5, value=GENERATE_KWARGS['repeat_penalty'], step=0.1, label='repeat_penalty', visible=do_sample),
    ]
    return generate_args


# компорнент переключения режима RAG
def get_rag_mode_component(db: Optional[VectorStore]) -> gr.Checkbox:
    value = visible = db is not None
    return gr.Checkbox(value=value, label='RAG Mode', scale=1, visible=visible)


# ================ ЗАГРУЗКА И ИНИЦИАЛИЗАЦИЯ МОДЕЛЕЙ ========================

# инициализация словаря с LLM моделью, флага поддержки системного промта моделью и логов загрузки
start_llm_model, start_support_system_role, load_log = load_llm_model(LLM_MODEL_REPOS[0], 'gemma-2-2b-it-Q8_0.gguf')
# инициализация словаря с моделью эмбедингов и логов загрузки
start_embed_model, load_log = load_embed_model(EMBED_MODEL_REPOS[0])


# ====================== ИНТЕРФЕЙС ПРИЛОЖЕНИЯ ============================

# тема оформления приложения
# theme = gr.themes.Monochrome()
theme = gr.themes.Base(primary_hue='green', secondary_hue='yellow', neutral_hue='zinc').set(
    loader_color='rgb(0, 255, 0)',
    slider_color='rgb(0, 200, 0)',
    body_text_color_dark='rgb(0, 200, 0)',
    button_secondary_background_fill_dark='green',
)
# настройка ширины окна приложения через CSS
css = '''.gradio-container {width: 60% !important}'''

# начало описания интерфейса приложения
with gr.Blocks(theme=theme, css=css) as interface:

    # ==================== СОСТОЯНИЯ ===============================

    # загруженные фрагменты текста (список объектов langchain Document)
    documents = gr.State([])
    # БД
    db = gr.State(None)
    # обогащенный контекстом промт пользователя
    user_message_with_context = gr.State('')
    # флаг поддержки системного промта моделью
    support_system_role = gr.State(start_support_system_role)

    # списки готовых репозиториев моделей HF
    llm_model_repos = gr.State(LLM_MODEL_REPOS)
    embed_model_repos = gr.State(EMBED_MODEL_REPOS)

    # словари с моделями LLM и эмбедингов
    llm_model = gr.State(start_llm_model)
    embed_model = gr.State(start_embed_model)



    # ==================== СТРАНИЦА БОТА =================================

    with gr.Tab(label='Chatbot'):
        with gr.Row():
            with gr.Column(scale=3):
                # окошко чат бота
                chatbot = gr.Chatbot(
                    type='messages',  # new in gradio 5+
                    show_copy_button=True,
                    bubble_full_width=False,
                    height=480,
                )
                # сообщение пользователя
                user_message = gr.Textbox(label='User')

                with gr.Row():
                    # кнопки отправить сообщение, стоп генерации и удалить историю чата
                    user_message_btn = gr.Button('Отправить')
                    stop_btn = gr.Button('Стоп')
                    clear_btn = gr.Button('Очистить чат')

            # ------------------ ПАРАМЕТРЫ ГЕНЕРАЦИИ -------------------------

            with gr.Column(scale=1, min_width=80):
                with gr.Group():
                    # длина истории которую будет учитывать модель
                    gr.Markdown('Размер истории')
                    history_len = gr.Slider(
                        minimum=0,
                        maximum=5,
                        value=0,
                        step=1,
                        info='Кол-во предыдущих сообщенией, учитываемых в истории',
                        label='history len',
                        show_label=False,
                        )

                    with gr.Group():
                        gr.Markdown('Параметры генерации')
                        # переключатель активации случайного семплирования при генерации текста моделью
                        do_sample = gr.Checkbox(
                            value=False,
                            label='do_sample',
                            info='Активация случайного семплирования',
                            )
                        # настройки семплирования
                        generate_args = get_generate_args(do_sample.value)
                        do_sample.change(
                            fn=get_generate_args,
                            inputs=do_sample,
                            outputs=generate_args,
                            show_progress=False,
                            )

        # переключатель включить или выключить режим RAG (даже если БД готова к работе RAG можно отключить)
        rag_mode = get_rag_mode_component(db=db.value)
        # число релевантных фргаментов текста для контекста и параметр score_threshold
        k, score_threshold = get_rag_settings(rag_mode=rag_mode.value, render=False)
        # нажатие переключателя режима RAG
        rag_mode.change(
            fn=get_rag_settings,
            inputs=[rag_mode],
            outputs=[k, score_threshold],
            )

        # отобразить в этом месте экрана параметры k и score_threshold
        with gr.Row():
            k.render()
            score_threshold.render()

        # ---------------- СИСТЕМНЫЙ ПРОМТ И СООБЩЕНИЕ ПОЛЬЗОВАТЕЛЯ ---------

        with gr.Accordion('Промт', open=True):
            # окошко для системного промта
            system_prompt = get_system_prompt_component(interactive=support_system_role.value)
            # итоговыый промт который подается в модель
            user_message_with_context = get_user_message_with_context(text='', rag_mode=rag_mode.value)

        # ------------------ КНОПКИ ОТПРАВИТЬ ОЧИСТИТЬ И СТОП ------------

        # нажатие Enter и кнопка отправить
        generate_event = gr.on(
            triggers=[user_message.submit, user_message_btn.click],
            fn=user_message_to_chatbot,
            inputs=[user_message, chatbot],
            outputs=[user_message, chatbot],
            queue=False,
        ).then(
            fn=update_user_message_with_context,
            inputs=[chatbot, rag_mode, db, k, score_threshold],
            outputs=[user_message_with_context],
        ).then(
            fn=get_user_message_with_context,
            inputs=[user_message_with_context, rag_mode],
            outputs=[user_message_with_context],
        ).then(
            fn=get_llm_response,
            inputs=[chatbot, llm_model, user_message_with_context, rag_mode, system_prompt,
                    support_system_role, history_len, do_sample, *generate_args],
            outputs=[chatbot],
        )

        # кнопка Стоп
        stop_btn.click(
            fn=None,
            inputs=None,
            outputs=None,
            cancels=generate_event,
            queue=False,
        )

        # кнопка Очистить чат
        clear_btn.click(
            fn=lambda: (None, ''),
            inputs=None,
            outputs=[chatbot, user_message_with_context],
            queue=False,
            )



    # ===================== СТРАНИЦА ЗАГРУЗКИ ФАЙЛОВ =========================

    with gr.Tab(label='Load documents'):
        with gr.Row(variant='compact'):
            # загрузка файлов и ссылок
            upload_files = gr.File(file_count='multiple', label='Загрузка текстовых файлов')
            web_links = gr.Textbox(lines=6, label='Ссылки на Web сайты или Ютуб')

        with gr.Row(variant='compact'):
            # параметры нарезки текста на фрагменты
            chunk_size = gr.Slider(50, 2000, value=500, step=50, label='Длина фрагментов')
            chunk_overlap = gr.Slider(0, 200, value=20, step=10, label='Длина пересечения фрагментов')

            # язык субтитров
            subtitles_lang = gr.Radio(
                SUBTITLES_LANGUAGES,
                value=SUBTITLES_LANGUAGES[0],
                label='Язык субтитров YouTube',
                )

        # кнопка загрузки документов и инициализации БД
        load_documents_btn = gr.Button(value='Загрузить документы и инициализировать БД')
        # статус прогресса загрузки файлов и инициализации БД
        load_docs_log = gr.Textbox(label='Статус загрузки и разделения документов', interactive=False)

        # главный цикл загрузки доков и инициализации ретривера
        load_documents_btn.click(
            fn=load_documents_and_create_db,
            inputs=[upload_files, web_links, subtitles_lang, chunk_size, chunk_overlap, embed_model],
            outputs=[documents, db, load_docs_log],
        ).success(
            fn=get_rag_mode_component,
            inputs=[db],
            outputs=[rag_mode],
        )

        gr.HTML("""<h3 style='text-align: center'>
        <a href="https://github.com/sergey21000/chatbot-rag" target='_blank'>GitHub Repository</a></h3>
        """)


    # ================= СТРАНИЦА ПРОСМОТРА ВСЕХ ДОКУМЕНТОВ =================

    with gr.Tab(label='View documents'):
        # кнопка и текстовое поле для отображения загруженных фрагментов текста
        view_documents_btn = gr.Button(value='Отобразить загруженные фрагменты')
        view_documents_textbox = gr.Textbox(
            lines=1,
            placeholder='Для просмотра фрагментов загрузите документы на вкладке Load documents',
            label='Загруженные фрагменты',
            )
        sep = '=' * 20
        # отображение загруженных фрагментов текста если они готовы
        view_documents_btn.click(
            lambda documents: f'\n{sep}\n\n'.join([doc.page_content for doc in documents]),
            inputs=[documents],
            outputs=[view_documents_textbox],
        )



    # ============== СТРАНИЦА ЗАГРУЗКИ GGUF МОДЕЛЕЙ =====================

    with gr.Tab('Load LLM model'):
        # окно добавления нового репозитория с HF
        new_llm_model_repo = gr.Textbox(
            value='',
            label='Добавить репозиторий',
            placeholder='Ссылка на репозиторий HF моделей в формате GGUF',
            )
        new_llm_model_repo_btn = gr.Button('Добавить репозиторий')

        # выбор репозитория HF из доступных
        curr_llm_model_repo = gr.Dropdown(
            choices=LLM_MODEL_REPOS,
            value=None,
            label='Репозиторий модели HF',
            )
        # выбор модели GGUF из выбранного репозитория
        curr_llm_model_path = gr.Dropdown(
            choices=[],
            value=None,
            label='Файл модели GGUF',
            )
        # кнопка загрузки и инициализации модели
        load_llm_model_btn = gr.Button('Загрука и инициализация модели')

        # статус загрузки модели
        load_llm_model_log = gr.Textbox(
            value=f'Модель {LLM_MODEL_REPOS[0]} загружена при старте приложения',
            label='Статус загрузки модели',
            lines=6,
            )

        # кнопка очистки директории ./models кроме текущей модели
        with gr.Group():
            gr.Markdown('Освободить место на диске путем удаления всех моделей кроме текущей выбранной')
            clear_llm_folder_btn = gr.Button('Очистить папку')

        # добавление нового репозитория и отображение статуса
        new_llm_model_repo_btn.click(
            fn=add_new_model_repo,
            inputs=[new_llm_model_repo, llm_model_repos],
            outputs=[curr_llm_model_repo, load_llm_model_log],
        ).success(
            fn=lambda: '',
            inputs=None,
            outputs=[new_llm_model_repo],
        )

        # получение списка моделей GGUF из выбранного репозитория
        curr_llm_model_repo.change(
            fn=get_gguf_model_names,
            inputs=[curr_llm_model_repo],
            outputs=[curr_llm_model_path],
        )

        # загрузка файла модели GGUF
        load_llm_model_btn.click(
            fn=load_llm_model,
            inputs=[curr_llm_model_repo, curr_llm_model_path],
            outputs=[llm_model, support_system_role, load_llm_model_log],
        ).success(
            fn=lambda log: log + get_memory_usage(),
            inputs=[load_llm_model_log],
            outputs=[load_llm_model_log],
        ).then(
            fn=get_system_prompt_component,
            inputs=[support_system_role],
            outputs=[system_prompt],
        )

        # очистка папки с моделями кроме текущей загруженной
        clear_llm_folder_btn.click(
            fn=clear_llm_folder,
            inputs=[curr_llm_model_path],
            outputs=None,
        ).success(
            fn=lambda model_path: f'Модели кроме {model_path} удалены',
            inputs=[curr_llm_model_path],
            outputs=None,
        )


    # ================= СТРАНИЦА ЗАГРУЗКИ ЭМБЕДИНГ МОДЕЛЕЙ =============

    with gr.Tab('Load embed model'):
        # окно добавления нового репозитория с HF
        new_embed_model_repo = gr.Textbox(
            value='',
            label='Добавить репозиторий',
            placeholder='Ссылка на репозиторий модели HF',
            )
        new_embed_model_repo_btn = gr.Button('Добавить репозиторий')

        # выбор репозитория HF из доступных
        curr_embed_model_repo = gr.Dropdown(
            choices=EMBED_MODEL_REPOS,
            value=None,
            label='Репозиторий модели HF',
            )

        # статус загрузки модели
        load_embed_model_btn = gr.Button('Загрука и инициализация модели')
        load_embed_model_log = gr.Textbox(
            value=f'Модель {EMBED_MODEL_REPOS[0]} загружена по умолчанию',
            label='Статус загрузки модели',
            lines=7,
            )
        with gr.Group():
            gr.Markdown('Освободить место на диске путем удаления всех моделей кроме текущей выбранной')
            clear_embed_folder_btn = gr.Button('Очистить папку')

        # добавление нового репозитория и отображение статуса
        new_embed_model_repo_btn.click(
            fn=add_new_model_repo,
            inputs=[new_embed_model_repo, embed_model_repos],
            outputs=[curr_embed_model_repo, load_embed_model_log],
        ).success(
            fn=lambda: '',
            inputs=None,
            outputs=new_embed_model_repo,
        )

        # загрузка модели эмбедингов
        load_embed_model_btn.click(
            fn=load_embed_model,
            inputs=[curr_embed_model_repo],
            outputs=[embed_model, load_embed_model_log],
        ).success(
            fn=lambda log: log + get_memory_usage(),
            inputs=[load_embed_model_log],
            outputs=[load_embed_model_log],
        )

        # очистка папки с моделями кроме текущей загруженной
        clear_embed_folder_btn.click(
            fn=clear_embed_folder,
            inputs=[curr_embed_model_repo],
            outputs=None,
        ).success(
            fn=lambda model_repo: f'Модели кроме {model_repo} удалены',
            inputs=[curr_embed_model_repo],
            outputs=None,
        )

# запуск в Colab
interface.launch(debug=True)

# запуск на локальной машине, удаленном сервере или в Docker
# interface.launch(server_name='0.0.0.0', server_port=7860)  # debug=True

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://efb275976fd581d286.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://efb275976fd581d286.gradio.live


# Docker

Содержимое всех файлов кроме `*docker*` такое же как в разделе `Полный код приложения` выше

Инструкции по образам Docker для `llama-cpp-python`  
https://github.com/abetlen/llama-cpp-python/tree/main/docker

Образы Nvidia CUDA  
https://catalog.ngc.nvidia.com/orgs/nvidia/containers/cuda/tags

Образы Pytorch от Nvidia  
https://catalog.ngc.nvidia.com/orgs/nvidia/containers/pytorch/tags

Образы Pytorch на Docker Hub  
https://hub.docker.com/r/pytorch/pytorch/tags


**Структура проекта:**

 - 📁 `embed_models`
 - 📁 `models`
 - `.dockerignore`
 - `Dockerfile-cpu`
 - `Dockerfile-cuda`
 - `requirements-base.txt`
 - `requirements-cpu.txt`
 - `requirements-cuda.txt`
 - `app.py`
 - `config.py`
 - `utils.py`



---

**Сборка образа и запуск контейнера:**

1) Сборка образа  

 - с поддержкой CPU
```
docker build -t chatbot-rag:cpu -f Dockerfile-cpu .
```
 - с поддержкой CUDA
```
docker build -t chatbot-rag:cuda -f Dockerfile-cuda .
```

2) Запуск контейнера на 7860 порту с пробросом папок через `docker volumes`
 - с поддержкой CPU
```
docker run -it -p 7860:7860 \
	-v ./embed_models:/app/embed_models \
	-v ./models:/app/models \
	--name chatbot-rag \
	chatbot-rag:cpu
```

 - с поддержкой CUDA
```
docker run -it --gpus all -p 7860:7860 \
	-v ./embed_models:/app/embed_models \
	-v ./models:/app/models \
	--name chatbot-rag \
	chatbot-rag:cuda
```

3) Перейти в браузере http://localhost:7860/ после того как появится надпись `Running on local URL:  http://0.0.0.0:7860`

---

**Содержимое `Dockerfile-cpu`**
```
FROM python:3.10
WORKDIR /app
RUN pip install --no-cache-dir llama_cpp_python==0.2.88
COPY requirements-cpu.txt requirements-base.txt .
RUN pip install --no-cache-dir -r requirements-cpu.txt
COPY app.py config.py utils.py .
EXPOSE 7860
CMD ["python3", "app.py"]
```
Здесь `llama_cpp_python` ставится заранее отдельно потому что иначе при установке через `whl` (как указано в `reuirements.txt`) при импорте возникала ошибка  
`Failed to load shared library '/usr/local/lib/python3.10/site-packages/llama_cpp/lib/libllama.so': libc.musl-x86_64.so.1: cannot open shared object fil`  
Можно объединить эти команды в одну для оптимизации размера образа (разницы в размере нет)

Несжатый образ CPU занимает 2.73 GB, сжатый 854 MB  

---

**Содержимое `Dockerfile-cuda` вариант через образы `nvidia/cuda`**
```
ARG CUDA_IMAGE="12.5.0-devel-ubuntu22.04"
FROM nvidia/cuda:${CUDA_IMAGE} AS builder

RUN apt-get update \
    && apt-get install -y python3 python3-pip \
    && apt-get clean && rm -rf /var/lib/apt/lists/*

ENV CUDA_DOCKER_ARCH=all
ENV GGML_CUDA=1
ENV CMAKE_ARGS="-DGGML_CUDA=on"

RUN pip install --no-cache-dir llama_cpp_python==0.2.88
COPY requirements-base.txt requirements-cuda.txt .
RUN pip install --no-cache-dir -r requirements-cuda.txt

ARG CUDA_IMAGE="12.5.0-runtime-ubuntu22.04"
FROM nvidia/cuda:${CUDA_IMAGE}

RUN apt-get update \
    && apt-get install -y python3 python3-pip \
    && apt-get clean && rm -rf /var/lib/apt/lists/*

ARG PYTHON_VERSION=3.10
COPY --from=builder /usr/local/lib/python${PYTHON_VERSION}/dist-packages /usr/local/lib/python${PYTHON_VERSION}/dist-packages

WORKDIR /app
COPY app.py config.py utils.py .
EXPOSE 7860
CMD ["python3", "app.py"]
```


Несжатый образ CUDA занимает 13.7 GB, сжатый 6.81 GB  

Для других версий CUDA заменить в `Dockerfile-cuda` соответствующие базовые образы вместо  
```
ARG CUDA_IMAGE="12.5.0-devel-ubuntu22.04"
ARG CUDA_IMAGE="12.5.0-runtime-ubuntu22.04"
```

---

**Содержимое `Dockerfile-cuda` вариант через образы `pytorch/pytorch`**
```
FROM pytorch/pytorch:2.4.1-cuda12.4-cudnn9-devel AS builder

ENV CUDA_DOCKER_ARCH=all
ENV GGML_CUDA=1
ENV CMAKE_ARGS="-DGGML_CUDA=on"

COPY requirements-base.txt .
RUN pip wheel --no-cache-dir --no-deps --wheel-dir /app/wheels llama_cpp_python==0.2.88 && \
    pip wheel --no-cache-dir --no-deps --wheel-dir /app/wheels -r requirements-base.txt

FROM pytorch/pytorch:2.4.1-cuda12.4-cudnn9-runtime
WORKDIR /app

COPY --from=builder /app/wheels /wheels
RUN pip install --no-cache-dir /wheels/*

COPY app.py config.py utils.py .

EXPOSE 7860
CMD ["python3", "app.py"]
```

Несжатый образ CUDA занимает 7.77 GB, сжатый 3.77 GB

# Прочее

Проверка моделей в формате GGUF через библиотеку `llama-cpp-python`

Функция для загрузки моделей через библиотеку `requests` с прогресс баром

In [ ]:
def download_file(file_url: str, file_path: Union[str, Path]) -> None:
    response = requests.get(file_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    chunk_size = 4096  # 4 Kb
    progress_bar = tqdm(desc='Загрузка файла GGUF', total=total_size, unit='iB', unit_scale=True)

    with open(file_path, 'wb') as file:
        for data in response.iter_content(chunk_size):
            file.write(data)
            progress_bar.update(len(data))
    # progress_bar.close()

Инийциализация модели ембедингов и LLM модели

In [ ]:
# инициализация модели эмбедингов
print('Инициализация модели эмбедингов ...')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embed_model_name = 'sentence-transformers/all-mpnet-base-v2'
embed_model_path = Path('embed_models')
embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_name,
    model_kwargs={'device': device},  # cpu cuda
    cache_folder=str(embed_model_path),
    )

# прямая ссылка на модель в формате GGUF
# llm_model_url = 'https://huggingface.co/bartowski/openchat-3.6-8b-20240522-GGUF/resolve/main/openchat-3.6-8b-20240522-IQ4_XS.gguf'
llm_model_url = 'https://huggingface.co/bartowski/gemma-2-2b-it-GGUF/resolve/main/gemma-2-2b-it-Q8_0.gguf'

# полный путь до модели в папке ./models
llm_model_path = Path('models') / llm_model_url.rsplit('/')[-1]
llm_model_path.parent.mkdir(exist_ok=True)

# если модель отсутствует в папке models то нужно ее загрузить - например через библиотеку wget
if not llm_model_path.is_file():
    if not str(llm_model_url).endswith('.gguf'):
        raise Exception('Ссылка на модель LLM должна быть прямой ссылкой на файл GGUF')
    else:
        print('Загрузка файла LLM модели ...')
        download_file(llm_model_url, llm_model_path)
        # model_path = wget.download(llm_model_url, out=str(llm_model_path))

# инициализация модели для генерации текста (установить нужный или поддерживаемый размер контекста n_ctx)
print('Инициализация LLM модели ...')
llm_model = Llama(model_path=str(llm_model_path), n_gpu_layers=-1, verbose=True)

# поддерживает ли модель системный промт
support_system_role = 'System role not supported' not in llm_model.metadata['tokenizer.chat_template']

# параметры генерации
# чтобы модель отвечала одинаково достаточно поставить top_k=1, top_p=0 и repeat_penalty=1 не зависимо от остальных параметров
GENERATE_KWARGS = dict(
    temperature=1,  # температура для софтмакса
    top_p=0.0,  # сумма вероятностей токенов из которых нужно выбирать следующий токен
    top_k=1,  # из скольки максимально вероятных токенов выбирать следующий токен
    repeat_penalty=1.0,  # штраф модели за повторения
    )

Инициализация модели эмбедингов ...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
llama_model_loader: loaded meta data with 39 key-value pairs and 288 tensors from model/gemma-2-2b-it-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Gemma 2 2b It
llama_model_loader: - kv   3:                           general.finetu

Инициализация LLM модели ...


llama_model_loader: - kv  23:                      tokenizer.ggml.tokens arr[str,256000]  = ["<pad>", "<eos>", "<bos>", "<unk>", ...
llama_model_loader: - kv  24:                      tokenizer.ggml.scores arr[f32,256000]  = [-1000.000000, -1000.000000, -1000.00...
llama_model_loader: - kv  25:                  tokenizer.ggml.token_type arr[i32,256000]  = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...
llama_model_loader: - kv  26:                tokenizer.ggml.bos_token_id u32              = 2
llama_model_loader: - kv  27:                tokenizer.ggml.eos_token_id u32              = 1
llama_model_loader: - kv  28:            tokenizer.ggml.unknown_token_id u32              = 3
llama_model_loader: - kv  29:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  30:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  31:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  32: 

Генерация текста моделью

Документация по методу `create_chat_completion`  
https://llama-cpp-python.readthedocs.io/en/latest/api-reference/#llama_cpp.Llama.create_chat_completion

In [ ]:
%%time

# входной запрос пользователя
user_message = 'Почему трава зеленая?'
# список с репликами юзера и бота (в данном примере одна)
messages = []
# формирование стандартного промта с запросом от пользователя
messages.append({'role': 'user', 'content': user_message})

# параметры генерации
# чтобы модель отвечала одинаково достаточно поставить top_k=1, top_p=0 и repeat_penalty=1 не зависимо от остальных параметров
GENERATE_KWARGS = dict(
    temperature=1,  # температура для софтмакса
    top_p=0.0,  # сумма вероятностей токенов из которых нужно выбирать следующий токен
    top_k=1,  # из скольки максимально вероятных токенов выбирать следующий токен
    repeat_penalty=1.0,  # штраф модели за повторения
    )

# создание объекта итератора для генерации текста
# при итерации по этому объекту в цикле она будет возвращать сгенерированный текст частями текста (токенами)
stream_response = llm_model.create_chat_completion(
    messages=messages,  # входной промт на который надо сгенерировать ответ
    stream=True,  # вернуть генератор
    **GENERATE_KWARGS,  # передача параметров генерации
    )

# пустую строку будем конкатенировать с токенами ответа модели
response_text = ''
# итерация и последовательная генерация текста моделью в цикле
for chunk in stream_response:
    # извлечение текущего сгенерированного токена
    token = chunk['choices'][0]['delta'].get('content')
    if token is not None:
        response_text += token
        print(token, end='')

Трава зеленая из-за пигмента **хлорофилла**. 

Вот как это работает:

* **Хлорофилл** - это пигмент, который содержится в хлоропластах, органеллах, расположенных в клетках растений. 
* **Хлорофилл** поглощает солнечный свет, особенно в красном и синем диапазоне, и отражает зеленый свет. 
* **Зеленый свет** - это то, что мы видим, когда трава отражает свет. 

Таким образом, трава кажется зеленой, потому что она отражает зеленый свет, который поглощается хлорофиллом. 



llama_print_timings:        load time =   14636.85 ms
llama_print_timings:      sample time =      45.09 ms /   149 runs   (    0.30 ms per token,  3304.50 tokens per second)
llama_print_timings: prompt eval time =   14631.99 ms /    15 tokens (  975.47 ms per token,     1.03 tokens per second)
llama_print_timings:        eval time =   76900.53 ms /   148 runs   (  519.60 ms per token,     1.92 tokens per second)
llama_print_timings:       total time =   92487.25 ms /   163 tokens


CPU times: user 1min 12s, sys: 465 ms, total: 1min 12s
Wall time: 1min 32s


Дополнительно

Проверка векторнгого поиска через методы `similarity_search`

Инициализация модели эмбедингов

In [ ]:
# model_name = 'inkoziev/sbert_pq'
# model_name = 'sergeyzh/LaBSE-ru-turbo'
model_name = 'intfloat/multilingual-e5-small'

embed_model = HuggingFaceEmbeddings(
    model_name=model_name,
    cache_folder='./embed_models',
    # encode_kwargs={'normalize_embeddings': True},  # доп нормализация векторов
    )

Проверка поиска

In [ ]:
from langchain_community.vectorstores.utils import DistanceStrategy

# документы для БД
docs = [Document(text) for text in ['зима', 'весна', 'лето']]

# инициализация БД
db = FAISS.from_documents(
    documents=docs,
    embedding=embed_model,
    normalize_L2=True,
    # distance_strategy=DistanceStrategy.COSINE,  # указать меру расстояния
    )

In [ ]:
# число от 0 до 1, чем больше тем более похож текст на запрос (на некоторых моделях это не так)
db.similarity_search_with_relevance_scores('f fd ffd &**& $##%^ &% %')

[(Document(metadata={}, page_content='лето'), 0.7641452874289455),
 (Document(metadata={}, page_content='зима'), 0.753656917736464),
 (Document(metadata={}, page_content='весна'), 0.7509914456695138)]

In [ ]:
# наоборот, чем меньше число тем более похож текст на запрос
db.similarity_search_with_score('f fd ffd &**& $##%^ &% %')

[(Document(metadata={}, page_content='лето'), 0.33354893),
 (Document(metadata={}, page_content='зима'), 0.34838173),
 (Document(metadata={}, page_content='весна'), 0.35215127)]

In [ ]:
db.similarity_search_with_relevance_scores('веселье', k=2, score_threshold=0.2)

[(Document(metadata={}, page_content='весна'), 0.8463678883149083),
 (Document(metadata={}, page_content='лето'), 0.8251721117582341)]

In [ ]:
db.similarity_search_with_score('веселье')

[(Document(metadata={}, page_content='весна'), 0.21726862),
 (Document(metadata={}, page_content='лето'), 0.24724397),
 (Document(metadata={}, page_content='зима'), 0.28593075)]

Изменение меры расстояния

In [ ]:
from langchain_community.vectorstores.utils import DistanceStrategy
db = FAISS.from_documents(
    documents=docs,
    embedding=embed_model,
    # distance_strategy=DistanceStrategy.COSINE,

    # https://github.com/langchain-ai/langchain/issues/9519
    distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT,
    )

In [ ]:
db.similarity_search_with_relevance_scores('f fd ffd &**& $##%^ &% %', k=2, score_threshold=0.1)

[(Document(metadata={}, page_content='лето'), 0.16677451133728027),
 (Document(metadata={}, page_content='зима'), 0.17419099807739258)]

In [ ]:
db.similarity_search_with_score('f fd ffd &**& $##%^ &% %')

[(Document(metadata={}, page_content='лето'), 0.8332255),
 (Document(metadata={}, page_content='зима'), 0.825809),
 (Document(metadata={}, page_content='весна'), 0.8239243)]

In [ ]:
db.similarity_search_with_relevance_scores('веселье', k=2, score_threshold=0.1)

[(Document(metadata={}, page_content='весна'), 0.108634352684021),
 (Document(metadata={}, page_content='лето'), 0.12362194061279297)]

In [ ]:
db.similarity_search_with_score('веселье')

[(Document(metadata={}, page_content='весна'), 0.89136565),
 (Document(metadata={}, page_content='лето'), 0.87637806),
 (Document(metadata={}, page_content='зима'), 0.85703456)]

In [ ]:
db.similarity_search_with_relevance_scores('грусть', k=2, score_threshold=0.1)

[(Document(metadata={}, page_content='лето'), 0.12584853172302246),
 (Document(metadata={}, page_content='весна'), 0.12598317861557007)]